In [1]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

In [2]:
# 国調人口を読み込む
tmp = spark.read.csv(r'input\平成27年国勢調査人口等基本集計（総務省統計局）.csv'
                        , header=True
                        , encoding='shift-jis')
tmp.show(5)

+----------+--------------+--------------+--------------+------+-------------------+--------------------------+------------------------------+------------------------------------+-----------------------+----------------------------------+------------------+----------------------------+--------------------------------+-------------------------------------+
|地域コード|地域識別コード|境域年次(2015)|境域年次(2000)|   _c4|人口　平成27年　(a)|人口　平成22年（組替） 2) |平成22年〜27年の人口増減数 2) |平成22年〜27年の人口増減率（％） 2) |面積（km2）　(b) 1) 2) |人口密度（1km2当たり）　(a)/(b) 2)|世帯数　平成27年　|世帯数　平成22年（組替） 2) |平成22年〜27年の世帯数増減数 2) |平成22年〜27年の世帯数増減率（％） 2)|
+----------+--------------+--------------+--------------+------+-------------------+--------------------------+------------------------------+------------------------------------+-----------------------+----------------------------------+------------------+----------------------------+--------------------------------+-------------------------------------+
|         0|             a|          2015|       

In [3]:
# 利用するカラムに絞る
tmp = tmp.where((tmp[1] == 'a') & (tmp[0] > 0)) \
         .withColumn('pop', tmp[5].astype('int'))
census = tmp.select(tmp[0], tmp[4], tmp.pop) \
            .toDF('area_cd', 'pref_nm', 'pop')
census.show(10)

+-------+-------+-------+
|area_cd|pref_nm|    pop|
+-------+-------+-------+
|   1000| 北海道|5381733|
|   2000| 青森県|1308265|
|   3000| 岩手県|1279594|
|   4000| 宮城県|2333899|
|   5000| 秋田県|1023119|
|   6000| 山形県|1123891|
|   7000| 福島県|1914039|
|   8000| 茨城県|2916976|
|   9000| 栃木県|1974255|
|  10000| 群馬県|1973115|
+-------+-------+-------+
only showing top 10 rows



In [4]:
# 都道府県コードを作成
udf_get_pref_cd = udf(lambda x: x.zfill(5)[:2], StringType())
census = census.withColumn('pref_cd', udf_get_pref_cd('area_cd')) \
               .select('pref_cd', 'pref_nm', 'pop')
census.show(10)

+-------+-------+-------+
|pref_cd|pref_nm|    pop|
+-------+-------+-------+
|     01| 北海道|5381733|
|     02| 青森県|1308265|
|     03| 岩手県|1279594|
|     04| 宮城県|2333899|
|     05| 秋田県|1023119|
|     06| 山形県|1123891|
|     07| 福島県|1914039|
|     08| 茨城県|2916976|
|     09| 栃木県|1974255|
|     10| 群馬県|1973115|
+-------+-------+-------+
only showing top 10 rows



In [5]:
# コロナ感染者情報を読み込む
tmp = spark.read.csv(r'input\corona-japan_20200414.csv'
                     , header=True
                     , encoding='utf-8')
tmp.show(5)

+----------+----------------+------------+------+------+
|都道府県名|ＰＣＲ検査陽性者|現在は入院等|退院者|死亡者|
+----------+----------------+------------+------+------+
|    東京都|            2171|        2095|    57|    19|
|    大阪府|             836|         683|   147|     6|
|  神奈川県|             572|         505|    55|    12|
|    千葉県|             474|         431|    39|     4|
|    埼玉県|             414|         383|    26|     5|
+----------+----------------+------------+------+------+
only showing top 5 rows



In [6]:
# 必要なカラムに絞る
covid19 = tmp.drop('現在は入院等', '退院者') \
             .toDF('pref_nm', 'pcr_positive', 'deceased')
covid19 = covid19.withColumn('pcr_positive', covid19.pcr_positive.astype('int')) \
                 .withColumn('deceased', covid19.deceased.astype('int'))
covid19.show(10)

+--------+------------+--------+
| pref_nm|pcr_positive|deceased|
+--------+------------+--------+
|  東京都|        2171|      19|
|  大阪府|         836|       6|
|神奈川県|         572|      12|
|  千葉県|         474|       4|
|  埼玉県|         414|       5|
|  兵庫県|         384|      14|
|  福岡県|         373|       0|
|  愛知県|         328|      23|
|  北海道|         272|      11|
|  京都府|         205|       3|
+--------+------------+--------+
only showing top 10 rows



In [7]:
cen_cov = census.join(covid19, on='pref_nm', how='left_outer') \
                .select('pref_cd', 'pref_nm', 'pop', 'pcr_positive', 'deceased') \
                .sort('pref_cd')

In [8]:
cen_cov.show(5)

+-------+-------+-------+------------+--------+
|pref_cd|pref_nm|    pop|pcr_positive|deceased|
+-------+-------+-------+------------+--------+
|     01| 北海道|5381733|         272|      11|
|     02| 青森県|1308265|          22|       0|
|     03| 岩手県|1279594|           0|       0|
|     04| 宮城県|2333899|          54|       0|
|     05| 秋田県|1023119|          15|       0|
+-------+-------+-------+------------+--------+
only showing top 5 rows



In [9]:
def get_rate(all_, some):
    if all_ == 0:
        return '0%'
    r = round((some / all_) * 100, 3)
    return f'{r}%'

udf_get_rate = udf(get_rate, StringType())

In [10]:
cen_cov = cen_cov.withColumn('rate_positive', udf_get_rate('pop', 'pcr_positive')) \
                 .withColumn('rate_deceased', udf_get_rate('pcr_positive', 'deceased'))
cen_cov.show()

+-------+--------+--------+------------+--------+-------------+-------------+
|pref_cd| pref_nm|     pop|pcr_positive|deceased|rate_positive|rate_deceased|
+-------+--------+--------+------------+--------+-------------+-------------+
|     01|  北海道| 5381733|         272|      11|       0.005%|       4.044%|
|     02|  青森県| 1308265|          22|       0|       0.002%|         0.0%|
|     03|  岩手県| 1279594|           0|       0|         0.0%|           0%|
|     04|  宮城県| 2333899|          54|       0|       0.002%|         0.0%|
|     05|  秋田県| 1023119|          15|       0|       0.001%|         0.0%|
|     06|  山形県| 1123891|          35|       0|       0.003%|         0.0%|
|     07|  福島県| 1914039|          38|       0|       0.002%|         0.0%|
|     08|  茨城県| 2916976|         110|       3|       0.004%|       2.727%|
|     09|  栃木県| 1974255|          36|       0|       0.002%|         0.0%|
|     10|  群馬県| 1973115|          91|       2|       0.005%|       2.198%|
|     11|  埼玉県| 

In [11]:
# 感染率ワースト5
cen_cov.sort(cen_cov.rate_positive.desc()).show(5)

+-------+-------+--------+------------+--------+-------------+-------------+
|pref_cd|pref_nm|     pop|pcr_positive|deceased|rate_positive|rate_deceased|
+-------+-------+--------+------------+--------+-------------+-------------+
|     13| 東京都|13515271|        2171|      19|       0.016%|       0.875%|
|     18| 福井県|  786740|          92|       2|       0.012%|       2.174%|
|     17| 石川県| 1154008|         121|       0|        0.01%|         0.0%|
|     27| 大阪府| 8839469|         836|       6|       0.009%|       0.718%|
|     12| 千葉県| 6222666|         474|       4|       0.008%|       0.844%|
+-------+-------+--------+------------+--------+-------------+-------------+
only showing top 5 rows



In [12]:
# 死亡率ワースト5
cen_cov.sort(cen_cov.rate_deceased.desc()).show(5)

+-------+-------+-------+------------+--------+-------------+-------------+
|pref_cd|pref_nm|    pop|pcr_positive|deceased|rate_positive|rate_deceased|
+-------+-------+-------+------------+--------+-------------+-------------+
|     23| 愛知県|7483128|         328|      23|       0.004%|       7.012%|
|     38| 愛媛県|1385262|          37|       2|       0.003%|       5.405%|
|     01| 北海道|5381733|         272|      11|       0.005%|       4.044%|
|     28| 兵庫県|5534800|         384|      14|       0.007%|       3.646%|
|     43| 熊本県|1786170|          28|       1|       0.002%|       3.571%|
+-------+-------+-------+------------+--------+-------------+-------------+
only showing top 5 rows

